# Feyisa automated water extraction index

From [Feyisa et al. (2014)](https://www.sciencedirect.com/science/article/pii/S0034425713002873)

In [2]:
import ee; ee.Initialize()

### Detect floods

In [3]:
scene = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044033_20170105')

# AWEI Function from Feyisa et al. (2014) to detect water
def feyisa(image):
    return image.expression("4 * (b('B3')-b('B6')) - (0.25 * b('B5') + 2.75 * b('B7'))").gt(0)

sceneFlood = feyisa(scene).rename('floodBinary')

### Remove clouds and cloud shadows

In [17]:
# Compute the bits we need to extract
def getQABits(image, start, end, newName):
    pattern = 0
    i = start
    for i in xrange(start, end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the band a new name
    return image.select([0], [newName])\
              .bitwiseAnd(pattern)\
              .rightShift(start);

# Cloud shadows QA (Landsat 8, T1 TOA)
def cloud_shadowsQA(image):
    QA = image.select(['BQA'])
    return getQABits(QA, 7, 8, 'cloud_shadows').eq(1)

# Clouds QA (Landsat 8, T1 TOA)
def cloudsQA(image):
    QA = image.select(['BQA'])
    return getQABits(QA, 4, 4, 'clouds').eq(0)

In [11]:
cs = cloud_shadowsQA(scene)
c = cloudsQA(scene)

sceneFlood = sceneFlood.updateMask(cs).updateMask(c)

32